# Introduction

Basic graph algotims

### BFS

In [1]:
import numpy as np
import networkx as nx

def BFS(G,start,end):
    
    queue = [start]
    visited = set([])
    while queue:
        print(queue)
        node = queue.pop(0)   #take from top
        if node == end:
            return True
        if node not in visited: 
            visited.add(node)
            new_neighbours = set(dict(G[node]).keys()) - visited
            for n in new_neighbours:
                queue.append(n)
    return False

G = nx.Graph()
edges = [(1,2),(1,3),(3,4),(3,5),(3,6),(1,6)]
G.add_edges_from(edges)
nx.draw(G)

BFS(G,1,5)

[1]
[2, 3, 6]
[3, 6]
[6, 4, 5, 6]
[4, 5, 6]
[5, 6]


True

In [2]:
def shortestPathBFS(G,start,end):
    
    queue = [(start,[start])]  #[ (node,[node1,node2,...]) ]
    visited = set([])
    while queue:
        (node,path) = queue.pop(0)   #take from top
        if node == end:
            return path
        if node not in visited: 
            visited.add(node)
            new_neighbours = set(dict(G[node]).keys()) - visited
            for n in new_neighbours:
                queue.append((n,path+[n]))
    return False

#Make graph
G = nx.Graph()
edges = [(1,2),(1,3),(3,4),(3,5),(3,6),(1,6)]
G.add_edges_from(edges)

path = shortestPathBFS(G,1,5)
path

[1, 3, 5]

### Find all cc's

In [3]:
import networkx as nx

def find_cc(G,start):
    queue = [start]
    visited = set([])
    while queue:
        node = queue.pop(0)
        if node not in visited:
            visited.add(node)
            neighbours = set(list(G[node].keys()))
            new_neighbours = neighbours - visited
            for n in new_neighbours:
                queue.append(n)
    return visited


def find_all_ccs(G):
    all_nodes = set(list(G.nodes()))
    ccs = []
    while all_nodes:
        start = all_nodes.pop()
        visited = find_cc(G,start)
        ccs.append(list(visited))
        all_nodes = all_nodes - visited
    return ccs
        

N,p = 20, 0.1
G = nx.erdos_renyi_graph(N,p)
ccs = find_all_ccs(G)
ccs

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19], [17]]

### DFS -- recursive

In [8]:
def dfs(G, start, goal, visited):
    if start == goal:
        return True
    else:
        new_visited = visited + [start]
        neighbours = list(G[start])
        for n in neighbours:
            if n not in new_visited:
                return dfs(G, n, goal, new_visited)
    return False


N,p = 20, 0.4
G = nx.erdos_renyi_graph(N,p)
dfs(G,5,10,[])

True

### Topological sort


In a directed acylic graph, every directed edge, (u,v), u comes before v in the ordering. This ordering is not unique. 

See https://stackoverflow.com/questions/47192626/deceptively-simple-implementation-of-topological-sorting-in-python

Below work for starting at one end

In [6]:
def dfs(G,start):
    
    result = [start]
    visited = set([])
    
    def helper(G,start):
        for n in set(G[start]) - visited:
            visited.add(n)
            result.append(n)
            helper(G,n)
            
    helper(G,5)
    return result


def topological_sort(G,start):
    
    result = []
    visited = set([])
    
    def helper(G,start):
        for n in set(G[start]) - visited:
            visited.add(n)
            helper(G,n)
        result.insert(0,start)
        
    helper(G,5)
    return result
                            

G = nx.DiGraph() 
G.add_edge(5, 2); 
G.add_edge(5, 0); 
G.add_edge(4, 0); 
G.add_edge(4, 1); 
G.add_edge(2, 3); 
G.add_edge(3, 1);

topological_sort(G,5)

[5, 2, 3, 1, 0]

In [7]:
def topological_sort(G):
    
    #visited list
    visited = set()
    stack = []
    
    def helper(G,node):
        for n in set(G[node]) - visited:
            visited.add(n)
            helper(G,n)
        stack.insert(0,node)
        
    for node in G.nodes():
        print('node = {}'.format(node))
        if node not in visited:
            helper(G,node)
    
    return stack

G = nx.DiGraph() 
G.add_edge(5, 2); 
G.add_edge(5, 0); 
G.add_edge(4, 0); 
G.add_edge(4, 1); 
G.add_edge(2, 3); 
G.add_edge(3, 1);
topological_sort(G)

node = 5
node = 2
node = 0
node = 4
node = 1
node = 3


[4, 5, 2, 3, 1, 0]

In [10]:
stack = [3,1,2]
stack.insert(0,4)

## Misc

In [1]:
def primeFactors(n):
    
    factors = []
    
    #divide by 2
    while n % 2 == 0:
        factors.append(2)
        n /= 2
    
    #n is now odd. So find smallest odd
    for i in range(3,int(n)+1,2):
        
        while n % i == 0:
            factors.append(i)
            n /= i
            
    return factors
            
factors = primeFactors(29)
factors

[29]